In [12]:
import torch
#import torchinfo
import timm
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from dataset import *

In [273]:
import torchmetrics

In [355]:
task = "multiclass"
top_k = 1
accuracy = torchmetrics.classification.Accuracy(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")
precision = torchmetrics.Precision(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")
recall = torchmetrics.Recall(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")
f1 = torchmetrics.F1Score(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")
stat = torchmetrics.StatScores(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")

In [356]:
preds = torch.tensor([[-0.0129, -0.3175, -0.0683,  0.2080,  0.1129,  0.0553,  0.0140,  0.0251,
         -0.1527,  0.0622],
        [ 0.1304,  0.1785,  0.0883,  0.5066, -0.1161, -0.1496, -0.2035,  0.0709,
          0.2420,  0.3981],
        [-0.0213, -0.0850, -0.1686,  0.1521,  0.2414, -0.0275,  0.0066,  0.1848,
         -0.1766,  0.0321],
        [ 0.3734,  0.3885, -0.2002, -0.3148, -0.2604,  0.3571, -0.7075, -0.2140,
          0.2051,  0.7275],
        [-0.1535, -0.1203, -0.0069,  0.2753,  0.3317, -0.3602, -0.1358,  0.1422,
         -0.0302,  0.3005],
        [ 0.2874,  0.1921,  0.2205,  0.3036, -0.1055, -0.1325,  0.0322,  0.4317,
          0.1032, -0.0067],
        [ 0.0414, -0.1034, -0.1293,  0.0489,  0.3273,  0.1023, -0.1438,  0.1175,
         -0.0843,  0.1121],
        [-0.1684, -0.0975, -0.1321,  0.1957,  0.1615,  0.1282, -0.2379,  0.0721,
         -0.1267,  0.1032],
        [ 0.0425, -0.1601, -0.0020,  0.1313,  0.1604,  0.1026,  0.0792,  0.1182,
         -0.0664,  0.0552],
        [-0.0258, -0.2268, -0.0416,  0.0605,  0.1985,  0.0211, -0.2180,  0.1641,
         -0.0347, -0.0319]]).to("cuda")
target = torch.tensor([3, 7, 0, 1, 3, 8, 9, 8, 3, 6]).to("cuda")
print(stat(preds, target))
print(accuracy(preds, target))
print(precision(preds, target))
print(recall(preds, target))
print(f1(preds, target))

tensor([ 1,  9, 81,  9, 10], device='cuda:0')
tensor(0.1000, device='cuda:0')
tensor(0.1000, device='cuda:0')
tensor(0.1000, device='cuda:0')
tensor(0.1000, device='cuda:0')


In [351]:
print(stat.compute())
print(accuracy.compute())
print(precision.compute())
print(recall.compute())
print(f1.compute())

tensor([ 4, 46, 44,  6, 10], device='cuda:0')
tensor(0.4000, device='cuda:0')
tensor(0.0800, device='cuda:0')
tensor(0.4000, device='cuda:0')
tensor(0.1333, device='cuda:0')


In [276]:
root_path = f"C:/Users/conon/Documents/Datasets/cifar10/"
train_path = os.path.join(root_path, "train")
val_path = os.path.join(root_path, "test")

train_labels_path = os.path.join(root_path, "train_labels.csv")
val_labels_path = os.path.join(root_path, "test_labels.csv")

In [138]:
train = datasets.CIFAR10(root=root_path, train=True, download=True)
val = datasets.CIFAR10(root=root_path, train=False, download=True)

for i, img in enumerate(train.data):
     np.save(os.path.join(root_path + "train", str(i)), img)
pd.DataFrame({"label": train.targets}).to_csv(train_labels_path, index=True)
for i, img in enumerate(val.data):
     np.save(os.path.join(root_path + "test", str(i)), img)
pd.DataFrame({"label": val.targets}).to_csv(val_labels_path, index=True)

Files already downloaded and verified
Files already downloaded and verified


In [26]:
timm.list_models("vgg*", pretrained=True)

['vgg11.tv_in1k',
 'vgg11_bn.tv_in1k',
 'vgg13.tv_in1k',
 'vgg13_bn.tv_in1k',
 'vgg16.tv_in1k',
 'vgg16_bn.tv_in1k',
 'vgg19.tv_in1k',
 'vgg19_bn.tv_in1k']

In [22]:
model = timm.create_model("fastvit_sa24", pretrained=True, num_classes=10, in_chans=3)

In [185]:
torchinfo.summary(model, input_size=[1, 3, 32, 32], depth=5)

Layer (type:depth-idx)                                                 Output Shape              Param #
FastVit                                                                [1, 10]                   --
├─Sequential: 1-1                                                      [1, 64, 8, 8]             --
│    └─MobileOneBlock: 2-1                                             [1, 64, 16, 16]           --
│    │    └─ConvNormAct: 3-1                                           [1, 64, 16, 16]           --
│    │    │    └─Conv2d: 4-1                                           [1, 64, 16, 16]           192
│    │    │    └─BatchNormAct2d: 4-2                                   [1, 64, 16, 16]           128
│    │    │    │    └─Identity: 5-1                                    [1, 64, 16, 16]           --
│    │    │    │    └─Identity: 5-2                                    [1, 64, 16, 16]           --
│    │    └─ModuleList: 3-2                                            --                    

In [23]:
param_name = [name for name,_ in model.named_parameters()] # All parameters name
layer_name = [name for name,_ in model.named_modules()] # All layers name

In [24]:
param_name

['stem.0.conv_kxk.0.conv.weight',
 'stem.0.conv_kxk.0.bn.weight',
 'stem.0.conv_kxk.0.bn.bias',
 'stem.0.conv_scale.conv.weight',
 'stem.0.conv_scale.bn.weight',
 'stem.0.conv_scale.bn.bias',
 'stem.1.conv_kxk.0.conv.weight',
 'stem.1.conv_kxk.0.bn.weight',
 'stem.1.conv_kxk.0.bn.bias',
 'stem.1.conv_scale.conv.weight',
 'stem.1.conv_scale.bn.weight',
 'stem.1.conv_scale.bn.bias',
 'stem.2.identity.weight',
 'stem.2.identity.bias',
 'stem.2.conv_kxk.0.conv.weight',
 'stem.2.conv_kxk.0.bn.weight',
 'stem.2.conv_kxk.0.bn.bias',
 'stages.0.blocks.0.token_mixer.norm.identity.weight',
 'stages.0.blocks.0.token_mixer.norm.identity.bias',
 'stages.0.blocks.0.token_mixer.mixer.identity.weight',
 'stages.0.blocks.0.token_mixer.mixer.identity.bias',
 'stages.0.blocks.0.token_mixer.mixer.conv_kxk.0.conv.weight',
 'stages.0.blocks.0.token_mixer.mixer.conv_kxk.0.bn.weight',
 'stages.0.blocks.0.token_mixer.mixer.conv_kxk.0.bn.bias',
 'stages.0.blocks.0.token_mixer.mixer.conv_scale.conv.weight',
 'st

In [16]:
model

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
      

In [17]:
requires_grad = False
for i, module in enumerate(model.named_parameters()):
    name, param = module
    if "head" in name:
        requires_grad = True
    param.requires_grad = requires_grad
for name, param in model.named_parameters():
    print(name, param.requires_grad)

conv_stem.weight False
bn1.weight False
bn1.bias False
blocks.0.0.conv_dw.weight False
blocks.0.0.bn1.weight False
blocks.0.0.bn1.bias False
blocks.0.0.se.conv_reduce.weight False
blocks.0.0.se.conv_reduce.bias False
blocks.0.0.se.conv_expand.weight False
blocks.0.0.se.conv_expand.bias False
blocks.0.0.conv_pw.weight False
blocks.0.0.bn2.weight False
blocks.0.0.bn2.bias False
blocks.0.1.conv_dw.weight False
blocks.0.1.bn1.weight False
blocks.0.1.bn1.bias False
blocks.0.1.se.conv_reduce.weight False
blocks.0.1.se.conv_reduce.bias False
blocks.0.1.se.conv_expand.weight False
blocks.0.1.se.conv_expand.bias False
blocks.0.1.conv_pw.weight False
blocks.0.1.bn2.weight False
blocks.0.1.bn2.bias False
blocks.1.0.conv_pw.weight False
blocks.1.0.bn1.weight False
blocks.1.0.bn1.bias False
blocks.1.0.conv_dw.weight False
blocks.1.0.bn2.weight False
blocks.1.0.bn2.bias False
blocks.1.0.se.conv_reduce.weight False
blocks.1.0.se.conv_reduce.bias False
blocks.1.0.se.conv_expand.weight False
blocks.1.0

In [18]:
pd.DataFrame.from_dict(model.default_cfg, orient="index")

,0
url,https://github.com/rwightman/pytorch-image-mod...
hf_hub_id,timm/efficientnet_b2.ra_in1k
architecture,efficientnet_b2
tag,ra_in1k
custom_load,False
input_size,"(3, 256, 256)"
test_input_size,"(3, 288, 288)"
fixed_input_size,False
interpolation,bicubic
crop_pct,1.0


In [70]:
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

In [71]:
transforms

Compose(
    Resize(size=284, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(256, 256))
    ToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)

In [74]:
model.default_cfg["input_size"][1:]

(256, 256)

In [82]:
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

transforms.Resize(size=256, interpolation=InterpolationMode.BICUBIC)


Resize(size=256, interpolation=bicubic, max_size=None, antialias=None)

In [10]:
# Custom final layer
num_in_features = model.get_classifier().in_features
model.head.fc = nn.Sequential(
    nn.BatchNorm1d(num_in_features),
    nn.Linear(in_features=num_in_features, out_features=512, bias=False),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.5),
    nn.Linear(in_features=512, out_features=10, bias=False)
)

In [114]:
model.eval()
model(torch.randn(1, 3, 256, 256)).shape

torch.Size([1, 10])

In [115]:
model.feature_info

[{'num_chs': 64, 'reduction': 4, 'module': 'stages.0'},
 {'num_chs': 128, 'reduction': 8, 'module': 'stages.1'},
 {'num_chs': 256, 'reduction': 16, 'module': 'stages.2'},
 {'num_chs': 512, 'reduction': 32, 'module': 'stages.3'}]

In [1]:
from minio import Minio

client = Minio("minio.pish.pstu.ru:80", access_key="U28XMjtvu9y6IdsThVBK", secret_key="tK0xuwoYxFBRSOLHy7dMdhD7ec6JZH7lSUNd5PXR", secure=False)

bucket_name = "ai-data"
folder_name = "Aircraft-FOD-DS-v2-Day-Binary-Reduced"
for item in client.list_objects(bucket_name, recursive=True):
    if folder_name in item.object_name:
        client.fget_object(bucket_name, item.object_name, item.object_name)

In [9]:
import os
len(os.listdir("./Aircraft-FOD-DS-v2-Day-Binary-Reduced/validation/0.normal"))

301

In [8]:
import os
len(os.listdir("./Aircraft-FOD-DS-v2-Day-Binary-Reduced/validation/1.abnormal"))

345

In [7]:
1357 + 1182

2539

In [10]:
301 + 345

646